# PyCity Schools Analysis

In [21]:
# Dependencies and Setup
from pathlib import Path
import pandas as pd

# File to Load
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how='left', on=['school_name', 'school_name'])

# Local Government Area Summary

In [22]:
# Calculate the Totals (Schools and Students)
school_count = school_data['school_name'].count()
student_count = student_data['student_name'].count()

# Calculate the Total Budget
total_budget = school_data['budget'].sum()

In [23]:
# Calculate the Average Scores
average_maths_score = student_data['maths_score'].mean()
average_reading_score = student_data['reading_score'].mean()

In [24]:
# Calculate the Percentage Pass Rates
passing_maths_count = school_data_complete[(school_data_complete["maths_score"] >= 50)].count()["student_name"]
passing_maths_percentage = passing_maths_count / float(student_count) * 100
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 50)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100

passing_maths_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 50) & (school_data_complete["maths_score"] >= 50)].count()["student_name"]

overall_passing_rate = passing_maths_reading_count / float(student_count) * 100

In [25]:
# Convert to DataFrame
area_summary = pd.DataFrame({"Total Schools": [school_count],
                            "Total Students": [student_count],
                            "Total Budget": [total_budget],
                            "Average Maths Score": [average_maths_score],
                            "Average Reading Score": [average_reading_score],
                            "% Passing Maths": [passing_maths_percentage],
                            "% Passing Reading": [passing_reading_percentage],
                            "% Overall Passing": [overall_passing_rate]})

# Formatting
area_summary["Total Students"] = area_summary["Total Students"].map("{:,}".format)
area_summary["Total Budget"] = area_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
area_summary

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


## School Summary

In [99]:
# Use the code provided to select the type per school from school_data
school_types = school_data.set_index(["school_name"])["type"]

# Calculate the total student count per school from school_data
# per_school_counts = school_data['size'].tolist()
per_school_counts = school_data['size']

# Calculate the total school budget and per capita spending per school from school_data
per_school_budget = school_data['budget']
per_school_capita = (per_school_budget / per_school_counts).to_list()

# Calculate the average test scores per school from school_data_complete
per_school_group = school_data_complete.groupby(['School ID'])

per_school_maths = per_school_group['maths_score'].mean().to_list()
per_school_reading = per_school_group['reading_score'].mean().to_list()

In [85]:
# Get the students who passed maths and passed reading by creating separate filtered DataFrames from school_data_complete.
student_passing_maths = school_data_complete.loc[school_data_complete['maths_score'] >= 50, ['School ID', 'student_name']].groupby(['School ID'])
# school_passing_maths = student_passing_maths['student_name'].count().to_list()
school_passing_maths = student_passing_maths['student_name'].count()

student_passing_reading = school_data_complete.loc[school_data_complete['reading_score'] >= 50, ['School ID', 'student_name']].groupby(['School ID'])
# school_passing_reading = student_passing_reading['student_name'].count().to_list()
school_passing_reading = student_passing_reading['student_name'].count()

# Get the students who passed both reading and maths in a separate DataFrame from school_data_complete.
student_passing_maths_and_reading = school_data_complete.loc[(school_data_complete["reading_score"] >= 50) & (school_data_complete["maths_score"] >= 50)].groupby(['School ID'])
# passing_maths_and_reading = student_passing_maths_and_reading['student_name'].count().to_list()
passing_maths_and_reading = student_passing_maths_and_reading['student_name'].count()

In [88]:
#  Calculate the Percentage Pass Rates
per_school_passing_maths = (school_passing_maths / per_school_counts * 100)
per_school_passing_reading = school_passing_reading / per_school_counts * 100
overall_passing_rate = passing_maths_and_reading / per_school_counts * 100

In [128]:
# Convert to DataFrame
per_school_summary = pd.DataFrame({'school_ID': school_data['School ID'].to_list(),
                                   'school_name': school_data['school_name'].to_list(),
                                   'School Type': school_data['type'].to_list(),
                                   'Total Students': per_school_counts,
                                   'Total School Budget': per_school_budget,
                                   'Per Student Budget': per_school_capita,
                                   'Average Maths Score': per_school_maths,
                                   'Average Reading Score': per_school_reading,
                                   '% Passing Maths': per_school_passing_maths,
                                   '% Passing Reading': per_school_passing_reading,
                                   '% Overall Passing': overall_passing_rate})
per_school_summary = per_school_summary.set_index(['school_name'])
del per_school_summary['school_ID']
per_school_summary = per_school_summary.sort_index()

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766


## Top Performing Schools (By % Overall Passing)

In [133]:
# Sort and show top five schools
top_schools = per_school_summary.sort_values(by='% Overall Passing', ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,90.797699,87.396849,79.419855


## Bottom Performing Schools (By % Overall Passing)

In [134]:
# Sort and show bottom five schools
bottom_schools = per_school_summary.sort_values(by='% Overall Passing', ascending=True)
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880


## Maths Scores by Year

In [ ]:
# Create data series of scores by year levels using conditionals
year_nine = school_data_complete[(school_data_complete["year"] == 9)]
year_ten = school_data_complete[(school_data_complete["year"] == 10)]
year_eleven = school_data_complete[(school_data_complete["year"] == 11)]
year_twelve = school_data_complete[(school_data_complete["year"] == 12)]

# Group each by school name
year_nine_scores =
year_ten_scores =
year_eleven_scores =
year_twelve_scores =

# Combine series into single DataFrame
maths_scores_by_year =

# Minor data wrangling
maths_scores_by_year.index.name = None

# Display the DataFrame
maths_scores_by_year
